In [1]:
import numpy as np
subh = np.loadtxt('../_data/subhalos.dat')

halo_mask, = np.where((subh[:,15]==0) & (subh[:,16]==0))

subh_name = subh[:,0][halo_mask]
subh_ind  = np.int_(subh_name)
# nsubh = len(subh_name)

print(subh_ind)

[ 0  1  3  4  5  6  7  9 20 21 22 27 28 31 33 35 36 37 38 39 41 42 45 46
 49 50 51 53 54 64 65 66 69 73 74 79 84 85 87 88 90 91 92 95 96]


### Saco las 10 particulas mas ligadas del halo

In [2]:
import matplotlib.pyplot as plt
import h5py
import numpy as np
import gc
import random

G = 4.299e-6

path = '/z/omarioni/gradients/_simulations/snap_127/'

for j in subh_ind[:1]:

    snap = h5py.File(path + 'subhalo_'+str('%03d'%j)+'.h5py', 'r')

    aexp = snap['/Time'][()]
    h    = snap['/h'][()]
    R200  = snap['/R200'][()]
    cen  = snap['/Center'][()]

    pdrk = snap['/Drk/Coordinates'][()]
    mdrk = snap['/Drk/Masses'][()]
    vdrk = snap['/Drk/Velocities'][()]    
    pot_drk = snap['/Drk/Potential'][()]   
    ID_drk  = snap['/Drk/IDs'][()]
    
    xdrk = pdrk[:,0]*aexp/h
    ydrk = pdrk[:,1]*aexp/h
    zdrk = pdrk[:,2]*aexp/h
    rdrk = np.sqrt(xdrk**2+ydrk**2+zdrk**2)
    
    v_x = vdrk[:,0] *np.sqrt(aexp)
    v_y = vdrk[:,1] *np.sqrt(aexp)
    v_z = vdrk[:,2] *np.sqrt(aexp)

    mdrk = mdrk*1e10/h
    pot_drk  = pot_drk/aexp
    
    mask, = np.where(rdrk<R200)
    
    e_kinetic = (1/2.)*(v_x[mask]**2 + v_y[mask]**2 + v_z[mask]**2)

    energy = pot_drk[mask] + e_kinetic
    
    sort = np.argsort(energy)
    particles = ID_drk[mask][sort][:10]
    
    print(particles)

[111442618 138005949 112832535 140178713 140160259 111469995 113389968
 113261481 111689283 113356056]


In [3]:
%%time
import gc
import h5py
import numpy as np

dir_path = '/store/clues/HESTIA/RE_SIMS/8192/GAL_FOR/17_11/output_2x2.5Mpc/groups_126/'

SubMass = []
SubPos  = []
SubR50  = []

for i in range(0,8):
    snap = h5py.File(dir_path + 'fof_subhalo_tab_126.'+ str('%s'%i) +'.hdf5','r')
    
    sub_mass = snap['Subhalo/SubhaloMass'][()]
    sub_pos  = snap['Subhalo/SubhaloPos'][()]
    sub_r50  = snap['Subhalo/SubhaloHalfmassRad'][()]

    SubMass.append(sub_mass)
    SubPos.append(sub_pos)
    SubR50.append(sub_r50)

SubMass = np.concatenate(SubMass)
SubPos  = np.concatenate(SubPos)
SubR50  = np.concatenate(SubR50)

gc.collect()

CPU times: user 59 ms, sys: 95.2 ms, total: 154 ms
Wall time: 163 ms


33

In [4]:
%%time
import gc
import joblib
import h5py
import pathlib
import numpy as np

dir_path = '/store/clues/HESTIA/RE_SIMS/8192/GAL_FOR/17_11/output_2x2.5Mpc/snapdir_126/'

DrkPos  = []
DrkID   = []

for i in range(0,8):
    snap = h5py.File(dir_path + 'snapshot_126.'+ str('%s'%i) +'.hdf5','r')
   
    pos_drk    = snap['PartType1/Coordinates'][()]
    ID_drk     = snap['PartType1/ParticleIDs'][()]
    
    DrkPos.append(pos_drk)
    DrkID.append(ID_drk)

a = snap.get('Header').attrs['Time']
    
DrkPos  = np.concatenate(DrkPos)
DrkID   = np.concatenate(DrkID)

gc.collect()

CPU times: user 29.6 s, sys: 24.8 s, total: 54.4 s
Wall time: 11.8 s


0

In [5]:
search = np.isin(DrkID, particles)

In [8]:
xdrk = DrkPos[search,0]*1000*a/h
ydrk = DrkPos[search,1]*1000*a/h
zdrk = DrkPos[search,2]*1000*a/h
rdrk = np.sqrt(xdrk**2+ydrk**2+zdrk**2)

SubPos_ = SubPos*1000*a/h

In [11]:
distance = np.zeros(len(SubMass))
for j in range(len(particles)):
    for i in range(len(SubMass)):
        distance[i] = np.sqrt((xdrk[j]-SubPos_[i,0])**2+(ydrk[j]-SubPos_[i,1])**2+(zdrk[j]-SubPos_[i,2])**2)
    print(np.argmin(distance))

0
0
0
0
0
0
0
0
0
0


In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
import pynbody

snap = np.loadtxt('/z/dalgorry/snapshots.txt', dtype='string')

fo=open('/z/dalgorry/datos_salida2/merger_tree-h1.dat','w')

s=pynbody.load('/srv/cosmdatc/clues/B64_WM3_186592/LG/GAS_SFR/4096_Gasoline/1024/WMAP3.CLUES.HR.001024')
h = s.halos()

time = s.properties['time'].in_units('Gyr')
print 'snap','1024','t=', time

aux = 1

fo.write(str('%s'% snap[len(snap)-1])+'\t'+
         str('%12.6f' %time)+'\t'+
         str('%02d'% aux)+'\n')
fo.flush()

h_aux = h[aux]
    
r200 = h_aux.properties['Rvir']
cen_pot = pynbody.analysis.halo.center(h_aux,mode='pot',retcen=True).in_units('kpc')
x_aux=((h_aux.dm['pos'].in_units('kpc'))[:,0] - cen_pot[0])
y_aux=((h_aux.dm['pos'].in_units('kpc'))[:,1] - cen_pot[1])
z_aux=((h_aux.dm['pos'].in_units('kpc'))[:,2] - cen_pot[2])
r_aux = np.sqrt(x_aux**2 + y_aux**2 + z_aux**2)

limit, = np.where(r_aux < r200)

e_pot = h_aux.dm['phi'].in_units('km**2 s**-2')

vel_aux = h_aux.dm['vel'].in_units('km s**-1')
mdrk_aux = h_aux.dm['mass'].in_units('Msol')

veloc,=np.where(r_aux < 0.05*r200)
vxcm_aux = sum(mdrk_aux[veloc]*vel_aux[:,0][veloc])/sum(mdrk_aux[veloc])
vycm_aux = sum(mdrk_aux[veloc]*vel_aux[:,1][veloc])/sum(mdrk_aux[veloc])
vzcm_aux = sum(mdrk_aux[veloc]*vel_aux[:,2][veloc])/sum(mdrk_aux[veloc])

vx_aux = vel_aux[:,0] - vxcm_aux
vy_aux = vel_aux[:,1] - vycm_aux
vz_aux = vel_aux[:,2] - vzcm_aux

e_cinetica = (1/2.)*(vx_aux**2 + vy_aux**2 + vz_aux**2)

energia= e_pot + e_cinetica

e_sort = np.sort(energia[limit])
e_indice = np.argsort(energia[limit])

ID = h_aux.dm['iord'][limit][e_indice][:10]

for j in range(len(snap)-2,-1,-1):
    
    s=pynbody.load('/srv/cosmdatc/clues/B64_WM3_186592/LG/GAS_SFR/4096_Gasoline/'+str('%s'%snap[j])+'/WMAP3.CLUES.HR.00'+str('%s'%snap[j]))
    h = s.halos()
    
    time = s.properties['time'].in_units('Gyr')       
    print 'snap', snap[j], 't=', time
    
    for i in range (1,100):
        k=0
        
        h_num = h[i]
        
        ID_num = h_num.dm['iord']

        for ID_part in ID:
            if ID_part in ID_num:
                k += 1
        if k > 6:
            break

    print k, 'halo','%s'%i
    
    aux = i
    
    fo.write(str('%s'% snap[j])+'\t'+
            str('%12.6f' %time)+'\t'+
            str('%02d'% aux)+'\n')
    fo.flush()
    
    h_aux = h[aux]
    
    r200 = h_aux.properties['Rvir']
    cen_pot = pynbody.analysis.halo.center(h_aux,mode='pot',retcen=True).in_units('kpc')
    x_aux=((h_aux.dm['pos'].in_units('kpc'))[:,0] - cen_pot[0])
    y_aux=((h_aux.dm['pos'].in_units('kpc'))[:,1] - cen_pot[1])
    z_aux=((h_aux.dm['pos'].in_units('kpc'))[:,2] - cen_pot[2])
    r_aux = np.sqrt(x_aux**2 + y_aux**2 + z_aux**2)
    
    limit, = np.where(r_aux < r200)
    
    e_pot = h_aux.dm['phi'].in_units('km**2 s**-2')
    
    vel_aux = h_aux.dm['vel'].in_units('km s**-1')
    mdrk_aux = h_aux.dm['mass'].in_units('Msol')
    
    veloc,=np.where(r_aux < 0.05*r200)
    vxcm_aux = sum(mdrk_aux[veloc]*vel_aux[:,0][veloc])/sum(mdrk_aux[veloc])
    vycm_aux = sum(mdrk_aux[veloc]*vel_aux[:,1][veloc])/sum(mdrk_aux[veloc])
    vzcm_aux = sum(mdrk_aux[veloc]*vel_aux[:,2][veloc])/sum(mdrk_aux[veloc])

    vx_aux = vel_aux[:,0] - vxcm_aux
    vy_aux = vel_aux[:,1] - vycm_aux
    vz_aux = vel_aux[:,2] - vzcm_aux

    e_cinetica = (1/2.)*(vx_aux**2 + vy_aux**2 + vz_aux**2)
    
    energia= e_pot + e_cinetica
    
    e_sort = np.sort(energia[limit])
    e_indice = np.argsort(energia[limit])
    
    ID = h_aux.dm['iord'][limit][e_indice][:10]  